In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import logging
import os
import warnings
import json

import numpy as np
import pandas as pd
import tensorflow as tf
# import tensorflow_hub as hub

from datetime import datetime
from keras.preprocessing import image
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from tensorflow.keras import layers

# from utils import *

In [3]:
os.chdir('F:\\UNIVERSITY\\UNIVERSITY_DOCUMENTS\\CS231\\doan_v2')

In [4]:
import numpy as np
import pandas as pd
import json
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
import time
import shutil
from zipfile import ZipFile

from PIL import Image

from keras.applications import VGG16
from keras.models import Model

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import EarlyStopping
from keras.models import Sequential, clone_model
from keras.layers import InputLayer, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, AveragePooling2D
from keras.layers import Dense, InputLayer, Flatten, Normalization, SimpleRNN, Embedding, LSTM, Bidirectional, GRU
# from keras.layers.normalization import LRN2D

from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from keras.optimizers import SGD, RMSprop, Adam

# **Function**

In [5]:
def load_json(path):
    with open(path, 'r') as file:
        json_file = json.load(file)
        return json_file


# **Load Repo**

In [6]:
# !git clone https://github.com/MrNquyen/Movie-Genres-CS231.git

In [7]:
# %cd Movie-Genres-CS231

# **Load Data**

In [8]:
train_path = 'data/train_new.json'
val_path = 'data/val.json'
test_path = 'data/test.json'

In [9]:
train_annot = load_json(train_path)
val_annot = load_json(val_path)
test_annot = load_json(test_path)

# **Train Test Split**

In [10]:
def create_df(annot):
    data = {
        'id': [],
        'name': [],
        'genre': [],
        'img_url': [],
        'img_path': [],
    }
    available_images = os.listdir('features_Resnet50')
    for key, val in annot.items():
        if f'{key}.npy' not in available_images:
            continue
        data['id'].append(key)
        for small_key in val.keys():
            data[small_key].append(val[small_key])
    return pd.DataFrame(data)


In [11]:
# X_train, y_train
train_df, val_df, test_df = create_df(train_annot), create_df(val_annot), create_df(test_annot)

In [12]:
X_train, y_train = train_df['id'], train_df['genre']
X_val, y_val = val_df['id'], val_df['genre']
X_test, y_test = test_df['id'], test_df['genre']

In [13]:
# y transform
classes_ = [
    'action', 'adventure', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'horror', 'mystery', 'thriller', 'romance', 'scifi', 'others'
]
label2id = {class_: i for i, class_ in enumerate(classes_)}

# Onehot Encode
mlb = MultiLabelBinarizer()
mlb.fit([classes_])

# Multilabel
y_train_new = mlb.transform(y_train).astype(float)
y_val_new = mlb.transform(y_val).astype(float)
y_test_new = mlb.transform(y_test).astype(float)

# **Load Image Tensor**

In [14]:
IMG_SIZE = 224
CHANNELS = 3

In [15]:
def parse_function(id, feature_dir='features_Resnet50'):
    npy_path = f'{feature_dir}/{id}.npy'
    image_features = np.load(npy_path)
    return image_features

In [16]:
from tqdm import tqdm
X_train_new = np.array([parse_function(id) for id in tqdm(X_train)])
X_test_new = np.array([parse_function(id) for id in tqdm(X_test)])

  0%|          | 0/4861 [00:00<?, ?it/s]

100%|██████████| 2511/2511 [00:49<00:00, 50.54it/s]


In [19]:
X_train_densenet = np.array([parse_function(id, feature_dir='features_Densenet169') for id in tqdm(X_train)])
X_test_densenet = np.array([parse_function(id, feature_dir='features_Densenet169') for id in tqdm(X_test)])

100%|██████████| 2511/2511 [00:52<00:00, 47.97it/s]


# **MODEL**

## ***VGG16***

In [30]:
model_vgg = Sequential([
    InputLayer((4096,)),
    Dense(
        units=1024,
        activation='relu',
    ),
    Dense(
        units=128,
        activation='relu',
    ),
    Dense(
        units=len(classes_),
        activation='sigmoid',
    ),
])
model_vgg.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 1024)           │     1,704,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 13)             │         1,677 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,837,837 (7.01 MB)

 Trainable params: 1,837,837 (7.01 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.

    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)

    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [59]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)

    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive

    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    y = tf.cast(y, tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [75]:
# Optimizer and Loss function
optimizer = Adam(learning_rate=0.01)
loss_fn = macro_soft_f1
metrics = [macro_f1]
# callback = EarlyStopping(monitor='val_loss', patience=3)

# Compile
model_vgg.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=metrics
)


In [61]:
# y_train_new.shape, X_train_new.shape

((1000, 13), (1000, 4096))

In [84]:
# Fit model
history_vgg = model_vgg.fit(
    X_train_new, y_train_new,
    batch_size=64,
    epochs=2,
    validation_data=(X_test_new, y_test_new),
    # validation_data=val_ds,
    # callbacks=[callback],
)

Epoch 1/2
76/76 ━━━━━━━━━━━━━━━━━━━━ 23s 296ms/step - loss: 0.6499 - macro_f1: 0.3501 - val_loss: 0.6985 - val_macro_f1: 0.2996
Epoch 2/2
76/76 ━━━━━━━━━━━━━━━━━━━━ 23s 304ms/step - loss: 0.6503 - macro_f1: 0.3497 - val_loss: 0.6985 - val_macro_f1: 0.2996


In [85]:
# prediction = []
# for batch in test_ds:
#     pred = model_vgg.predict(batch[0])
#     prediction.append(pred)

# prediction_concat = np.concatenate(prediction)
pred = model_vgg.predict(X_test_new)

79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step


In [ ]:
# def thresholding_results(y_logits, thres=0.5):
#     y_logits_copy = y_logits.copy()
#     y_logits_copy[y_logits_copy >= thres] = 1
#     y_logits_copy[y_logits_copy < thres] = 0
#     return y_logits_copy

# y_pred = thresholding_results(prediction_concat)

In [90]:
# %cd /content/drive/MyDrive/UNIVERSITY/SUBJECT/CS231/doan/transfer_learning
os.chdir('F:\\UNIVERSITY\\UNIVERSITY_DOCUMENTS\\CS231\\doan_v2\\evaluation')
np.save('vgg_layer', pred)

In [ ]:
np.save('y_test', y_test_new)

## ***DENSE***

In [245]:
X_train_new.shape

(4863, 4096)

In [272]:
model_dense = Sequential([
    InputLayer(X_train_new[0].shape),
    Dense(
        units=1024,
        activation='relu',
    ),
    Dense(
        units=128,
        activation='relu',
    ),
    Dense(
        units=len(classes_),
        activation='sigmoid',
    ),
])

In [252]:
model_dense.summary()

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_58 (Dense)                │ (None, 1024)           │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 13)             │         1,677 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,328,205 (16.51 MB)

 Trainable params: 4,328,205 (16.51 MB)

 Non-trainable params: 0 (0.00 B)

In [253]:
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.

    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)

    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [254]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)

    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive

    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    y = tf.cast(y, tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [273]:
# Optimizer and Loss function
optimizer = Adam(learning_rate=0.0001)
loss_fn = macro_soft_f1
metrics = [macro_f1]
callback = EarlyStopping(monitor='val_loss', patience=3)

# Compile
model_dense.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=metrics
)

In [274]:
# Fit model
history_dense = model_dense.fit(
    X_train_new, y_train_new,
    batch_size=64,
    epochs=10,
    validation_data=(X_test_new, y_test_new),
    # validation_data=val_ds,
    callbacks=[callback],
)

Epoch 1/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - loss: 0.6655 - macro_f1: 0.3293 - val_loss: 0.6984 - val_macro_f1: 0.2995
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.6504 - macro_f1: 0.3509 - val_loss: 0.6937 - val_macro_f1: 0.3082
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.6429 - macro_f1: 0.3593 - val_loss: 0.6984 - val_macro_f1: 0.2994
Epoch 4/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 0.6304 - macro_f1: 0.3731 - val_loss: 0.6884 - val_macro_f1: 0.3112
Epoch 5/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.6288 - macro_f1: 0.3767 - val_loss: 0.6883 - val_macro_f1: 0.3120
Epoch 6/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.6272 - macro_f1: 0.3758 - val_loss: 0.6943 - val_macro_f1: 0.3049
Epoch 7/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.6262 - macro_f1: 0.3748 - val_loss: 0.6876 - val_macro_f1: 0.3105
Epoch 8/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - loss: 0.6219 - macro_f1: 0.3823 - val_loss: 0.6888 - val_m

In [275]:
# prediction = []
# for batch in test_ds:
#     pred = model_vgg.predict(batch[0])
#     prediction.append(pred)

# prediction_concat = np.concatenate(prediction)
pred = model_dense.predict(X_test_new)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [277]:
# %cd /content/drive/MyDrive/UNIVERSITY/SUBJECT/CS231/doan/transfer_learning
os.chdir('F:\\UNIVERSITY\\UNIVERSITY_DOCUMENTS\\CS231\\doan_v2\\evaluation')
np.save('resnet_layer', pred)
np.save('y_test_resnet_layer', y_test_new)

# **RESNET50**

In [ ]:
# Do the same change the feature_dir to load Input Feature